In [1]:
import numpy as np
import re
from collections import Counter, namedtuple

In [2]:
Bot = namedtuple('Bot', 'x y z r')
LINE_RE = re.compile(r'pos=< *(-?\d+), *(-?\d+), *(-?\d+)>, *r=*(-?\d+)')

In [3]:
bots = []
with open('input.txt') as f:
    for line in f:
        vals = list(map(int, LINE_RE.findall(line)[0]))
        bots.append(Bot(x=vals[0], y=vals[1], z=vals[2], r=vals[3]))

In [4]:
with open('coords.txt', 'w') as f:
    for i, bot in enumerate(bots, 1):
        print('{} {} {} {} {}'.format(i, bot.x, bot.y, bot.z, bot.r), file=f)

In [5]:
# Is there a unique maximum bot? Yes
Counter(Counter(bot.r for bot in bots).values())

Counter({1: 1000})

In [6]:
max_bot = sorted(bots, key=lambda bot: bot.r)[-1]

In [7]:
def dist(bot1, bot2):
    return abs(bot1.x - bot2.x) + abs(bot1.y - bot2.y) + abs(bot1.z - bot2.z)

In [8]:
bots_in_range = [bot for bot in bots if dist(bot, max_bot) <= max_bot.r]

In [9]:
print('The answer to part 1 is:', len(bots_in_range))

The answer to part 1 is: 499


### Part 2

This turned out to be a horrendous problem that requires some branch and bound action due to the fact that we're in 3d and so Helly's theorem only gets you to _quadruples_ of intersections. Sigh. So below is a an (integer) linear program that will do the trick, but it runs too slow with the base Python stuff and my tiny box. So I stole a solution using Microsoft's Z3 solver from [this thread](https://www.reddit.com/r/adventofcode/comments/a8s17l/2018_day_23_solutions/) and ran it on a big ole AWS box because this is annoying.

The linear program in full is as follows: Let $x = (x_1, x_2, x_3)$ be the point we're searching for. Then the $L^1$ balls are bounded by $Ax \leq Ac_i + r_i$ where $c_i$ is the center of the $i$th ball and $r_i$ is its radius. Note that if we add a slack variable $y_i \in \{0, 1\}$ then we can _conditionally_ expand all radii by $M$ by replacing these constraints with $Ax \leq Ac_i + r_i + y_iM$. Thus, if we take $M \gg 0$ (e.g., the maximum $L^1$ distance from the origin of any point in any ball, which is just $2\max \| c_i \|_1 + r_i$, then we are good.

Then we simply find $(x, y)$ such that $\sum y_i$ is minimized and subject to the constraints $x$ is integral, $y$ is binary, and

$$ Ax \leq Ac_i + r + 2 y_i \max_j (\|c_j\|_1 + r_j) $$

In [10]:
# The basic shape of the L1 ball
A = np.array([
    [1, 1, 1], 
    [1, 1, -1],
    [1, -1, 1], 
    [1, -1, -1],
    [-1, 1, 1], 
    [-1, 1, -1],
    [-1, -1, 1], 
    [-1, -1, -1]])

# Max additional radius
M = 2 * max(abs(bot.x) + abs(bot.y) + abs(bot.z) + bot.r for bot in bots)

# A ball for every bot
bot_A = np.vstack([A] * len(bots))

# Recenter the balls and give them the desired radius
def b_from_bot(bot):
    return A @ np.array([bot.x, bot.y, bot.z]).T + bot.r

bot_b = np.array([b for bot in bots for b in b_from_bot(bot)])

# Add optional slack variables: We'll *minimize* the number that is used!
slack = np.zeros((bot_A.shape[0], len(bots)))
for i in range(slack.shape[0]):
    slack[i, i // A.shape[0]] = M
    
# Stack all of our constraints together
with_slack_A = np.hstack([bot_A, -slack])
bound_slack = np.hstack([np.zeros((len(bots), bot_A.shape[1])), -np.eye(len(bots))])
all_A = np.vstack([with_slack_A, bound_slack])
all_b = np.hstack([bot_b, np.zeros(len(bots))])

# Minimize the number of slack variables that are used
c = np.hstack([np.zeros(3), np.ones(1000)])

# Do the program. This turns out to be hard in python....
# linprog(c, all_A, all_b)

In [22]:
# Since linear programs are hard in python, we just output this to 

with open('dec23.mod', 'w') as f:
    print("""\
# A big number
param M;

# Balls: Index, x, y, z, radius
set I, dimen 5;

# Indices
set J := setof{(i, cx, cy, cz, r) in I} i;

# Location of point in intersection
set ddims := {1, 2, 3};
var x{ddims}, integer;

# Positive variables are *not* in the intersection
var slack{J}, binary;

minimize obj :
  sum{(i, cx, cy, cz, r) in I} slack[i];

s.t. plane1 {(i, cx, cy, cz, r) in I} :
  (x[1] - cx) + (x[2] - cy) + (x[3] - cz) <= r + M * slack[i];

s.t. plane2 {(i, cx, cy, cz, r) in I} :
  (x[1] - cx) + (x[2] - cy) - (x[3] - cz) <= r + M * slack[i];

s.t. plane3 {(i, cx, cy, cz, r) in I} :
  (x[1] - cx) - (x[2] - cy) + (x[3] - cz) <= r + M * slack[i];

s.t. plane4 {(i, cx, cy, cz, r) in I} :
  (x[1] - cx) - (x[2] - cy) - (x[3] - cz) <= r + M * slack[i];

s.t. plane5 {(i, cx, cy, cz, r) in I} :
  -(x[1] - cx) + (x[2] - cy) + (x[3] - cz) <= r + M * slack[i];

s.t. plane6 {(i, cx, cy, cz, r) in I} :
  -(x[1] - cx) + (x[2] - cy) - (x[3] - cz) <= r + M * slack[i];

s.t. plane7 {(i, cx, cy, cz, r) in I} :
  -(x[1] - cx) - (x[2] - cy) + (x[3] - cz) <= r + M * slack[i];

s.t. plane8 {(i, cx, cy, cz, r) in I} :
  -(x[1] - cx) - (x[2] - cy) - (x[3] - cz) <= r + M * slack[i];

solve;

printf "The central coordinate is:\\n";
printf "%i ", x[1];
printf "%i ", x[2];
printf "%i ", x[3];
printf "\\n";
printf "The answer to part 2 is: ";
printf "%i", x[1] + x[2] + x[3];

data;

# Size of the knapsack""", file=f)
    print('param M :=', M, ';', file=f)
    print()

    print("""\
# Items: index, size, profit
set I :=""", file=f)

    for i, bot in enumerate(bots, 1):
        fstr = '  {} {} {} {} {}'
        if i == len(bots):
            fstr += ';'
        print(fstr.format(i, bot.x, bot.y, bot.z, bot.r), file=f)

    print()
    print('end;', file=f)


In [11]:
from z3 import If, Int, Optimize

def zabs(x):
    return If(x >= 0,x,-x)

(x, y, z) = (Int('x'), Int('y'), Int('z'))
in_ranges = [Int('in_range_' + str(i)) for i in range(len(bots))]
range_count = Int('sum')
o = Optimize()

for in_range, bot in zip(in_ranges, bots):
    o.add(in_range == If(zabs(x - bot.x) + zabs(y - bot.y) + zabs(z - bot.z) <= bot.r, 1, 0))

o.add(range_count == sum(in_ranges))
dist_from_zero = Int('dist')

o.add(dist_from_zero == zabs(x) + zabs(y) + zabs(z))

h1 = o.maximize(range_count)
h2 = o.minimize(dist_from_zero)

print(o.check())
print('The answer to part 2 is:', o.lower(h2))

sat
b 80162663 80162663
